In [258]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from collections import deque 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [259]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [260]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [261]:
X,y = make_classification(1000,150,random_state=50,weights=[0.9])
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=50)

In [262]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred_prob = lr.predict_proba(X_test)

In [263]:
def rew(state):
    y_pred = np.where(pred_prob[:,1]>state,1,0)
    return f1_score(y_test, y_pred)

In [264]:
#  actions = {0 : -0.4, 1 : -0.1, 2 : -0.01, 3 : -0.001, 4 : 0, 5 : 0.001,  6 : 0.01, 7 : 0.1, 8: 0.4}
#  len(actions)

In [265]:
 actions = {0 : -0.1, 1 : -0.01, 2 : -0.001, 3 : 0, 4 : 0.001, 5 : 0.01,  6 : 0.1}
 len(actions)

7

In [266]:
# def step(action, state):
#     state = state[0]
    
#     r = rew(state)
   
#     next_state = state + actions[action]

#     return r, np.array([next_state])

In [267]:
# def step(action, state):
#     state = state[0]
#     next_state = state + actions[action]
#     r = rew(next_state)
#     return r, np.array([next_state])

In [268]:
def step(action, state):
    state = state[0]

    r_state = rew(state)
    next_state = state + actions[action]
    r_next = rew(next_state)

    if r_next > r_state:
        r = 1
    elif next_state == state:
        r = -1
    else:
        r = 0
        
    return r, np.array([next_state])

In [269]:
%run -i ./DQN_Class.py
gamma=0.5
epsilon=.8
numberEpisodes= 120
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


In [270]:
LearningQDeep.epsilon

0.8

In [271]:
%%time
LearningQDeep.trainingEpisodes()

Simulating episode 0
	 Max of rewards 1
	 Min state 0.8248719510909169
	 Max state 0.9248719510909169
Simulating episode 1
	 Max of rewards 1
	 Min state 0.8496690145708446
	 Max state 0.9596690145708446
Simulating episode 2
	 Max of rewards 0
	 Min state 0.0943053499405847
	 Max state 0.0943053499405847
Simulating episode 3
	 Max of rewards 1
	 Min state 0.72547728529314
	 Max state 0.92547728529314
Simulating episode 4
	 Max of rewards 1
	 Min state 0.7151199062865149
	 Max state 0.9161199062865149
Simulating episode 5
	 Max of rewards 0
	 Min state 0.16028127376297796
	 Max state 0.16128127376297796
Simulating episode 6
	 Max of rewards 1
	 Min state 0.20165375304239286
	 Max state 0.8306537530423928
Simulating episode 7
	 Max of rewards 1
	 Min state 0.14581537595723293
	 Max state 0.24581537595723293
Simulating episode 8
	 Max of rewards 1
	 Min state 0.4147518142739338
	 Max state 0.6257518142739338
Simulating episode 9
	 Max of rewards 1
	 Min state 0.03833939244289086
	 Max sta

In [126]:
LearningQDeep.mainNetwork.save('Trained_model.h5')

In [272]:
loaded_model = LearningQDeep.mainNetwork

In [273]:
actions_str = {0 : '----', 1 : '---', 2 : '--', 3 : '-', 4 : '0', 5 : '+', 6 : '++',  7 : '+++',8: '++++'}

In [277]:
def rew(state):
    y_pred = np.where(pred_prob[:,1]>state,1,0)
    return precision_score(y_test, y_pred)

In [278]:
import plotly.express as px

In [279]:
fig = px.line(x = np.arange(0,1,0.001), y = [rew(t) for t in np.arange(0,1,0.001)])
fig.show()

In [280]:
terminalState=False
# currentState = np.array([np.random.uniform(0,1)])
currentState = np.array([0.01])
terminated = False
# print(currentState)
# while not terminated:
for _ in range(50):
    # get the Q-value (1 by 2 vector)
    Qvalues=loaded_model.predict(currentState.reshape(1,1))
    # select the action that gives the max Qvalue
    action=np.random.choice(np.where(Qvalues[0,:]==np.max(Qvalues[0,:]))[0])
    # print(action)

    # if you want random actions for comparison
    #action = env.action_space.sample()
    # apply the action
    r, currentState = step(action, currentState)

    if r<0:
        terminated = True
    # if currentState<=0:
        # terminated = True

    print(actions_str[action], r,currentState)
    # sum the rewards
    # sumObtainedRewards+=currentReward

+ 1 [0.02]
+ 1 [0.03]
+ 1 [0.04]
+ 0 [0.05]
+ 0 [0.06]
+ 0 [0.07]
+ 1 [0.08]
+ 1 [0.09]
+ 1 [0.1]
+ 0 [0.11]
+ 0 [0.12]
+ 1 [0.13]
+ 1 [0.14]
+ 0 [0.15]
+ 0 [0.16]
+ 1 [0.17]
+ 1 [0.18]
+ 1 [0.19]
+ 0 [0.2]
+ 1 [0.21]
+ 1 [0.22]
+ 0 [0.23]
+ 0 [0.24]
+ 0 [0.25]
+ 0 [0.26]
+ 0 [0.27]
++ 0 [0.37]
++ 1 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]
++ 1 [0.57]
---- 0 [0.47]


In [140]:
currentState

array([0.63094663])